markstro
3/27/2020

This notebook is for estimating the parameter "dday_slope" for GF v1.1



Compute the daily radadj value for each HRU for each day

In [1]:
import numpy as np
#import pandas as pd
import fiona 
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Next blocks load in the monthly solrad targets from NREL Direct Normal Irradiance from http://nrel.gov/gis/solar.html

In [2]:
nrel_shapefile_fn = "c:/Users/markstro/work1.1/GIS/nhm_shapefiles/nhm_hru_gf1_1_nrel_solrad.shp"

In [3]:
# Read in the monthly short wave data from NREL

hrus = gpd.read_file(nrel_shapefile_fn)

In [4]:
hrus.head()

,OBJECTID,nhru_v11,hru_segmen,nhm_id,hru_id_nat,Version,Shape_Leng,Shape_Area,dni_jan_me,dni_feb_me,...,dni_apr_me,dni_may_me,dni_jun_me,dni_jul_me,dni_aug_me,dni_sep_me,dni_oct_me,dni_nov_me,dni_dec_me,geometry
0,1.0,76127,40038,76128,76128,1,92601.509382,1.881513e+08,182.201489,183.518720,...,207.460355,214.931469,256.187361,261.887199,257.431331,232.848366,229.749484,196.082145,166.266536,"POLYGON ((-105544.567 804074.976, -105542.972 ..."
1,2.0,76147,40038,76148,76148,1,60460.683381,4.416189e+07,182.087354,182.827321,...,207.058857,213.263005,254.263475,261.069763,255.607433,231.055495,227.661986,193.967239,164.803657,"POLYGON ((-97185.217 806355.005, -97184.805 80..."
2,3.0,76170,40021,76171,76171,1,62333.253245,7.337575e+07,183.556248,184.545532,...,207.761440,214.004899,256.235609,263.360321,258.373488,232.371546,228.184034,194.836239,165.232111,"MULTIPOLYGON (((-106590.453 815804.909, -10646..."
3,4.0,76172,40019,76173,76173,1,36798.115971,3.627274e+07,184.019112,185.748908,...,209.134271,213.834536,256.499269,263.734024,259.503738,232.098319,228.484543,195.671808,164.857940,"POLYGON ((-106762.641 815925.020, -106754.831 ..."
4,5.0,76181,40019,76182,76182,1,31441.172667,1.564925e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-109785.311 816675.047, -109784.942 ..."


In [5]:
hrus.tail()

,OBJECTID,nhru_v11,hru_segmen,nhm_id,hru_id_nat,Version,Shape_Leng,Shape_Area,dni_jan_me,dni_feb_me,...,dni_apr_me,dni_may_me,dni_jun_me,dni_jul_me,dni_aug_me,dni_sep_me,dni_oct_me,dni_nov_me,dni_dec_me,geometry
114953,114954.0,57964,31028,57965,57965,1,144351.031108,2.550743e+08,114.202190,146.516835,...,222.997086,240.846026,259.356323,306.073085,269.155830,239.265499,163.882154,127.888121,107.309903,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,114955.0,64080,28886,64081,64081,1,79361.919882,1.102709e+08,116.910396,155.363572,...,211.189573,218.493133,262.913941,334.356097,293.992769,252.081908,174.155373,133.079809,103.451310,"POLYGON ((-1284185.000 2658495.000, -1284185.0..."
114955,114956.0,64150,28866,64151,64151,1,95810.806831,1.884409e+08,100.200522,130.797801,...,221.951858,237.911750,274.295443,350.563980,296.977183,250.828283,176.225818,124.652525,94.911594,"POLYGON ((-1347645.097 2651834.819, -1347614.8..."
114956,114957.0,65633,31412,65634,65634,1,83210.535851,7.387074e+07,106.988632,148.589287,...,223.445490,243.563589,281.699072,354.641561,307.093563,247.162792,177.625283,130.361148,109.739219,"POLYGON ((-1017435.000 2869395.054, -1017435.0..."
114957,114958.0,18843,10081,18844,18844,1,198719.274301,2.223473e+08,128.077367,176.159291,...,243.767839,227.622825,238.529215,268.132795,244.427464,196.997736,137.297803,98.566210,104.569645,"POLYGON ((437685.115 2806784.914, 437685.231 2..."


In [6]:
# The HRU IDs for GF v1.1 are in the "nhru_v11" column

hru_ids = hrus['nhru_v11'].values
nhru = len(hru_ids)
print(min(hru_ids), max(hru_ids), nhru)
print(hru_ids)

1 114958 114958
[76127 76147 76170 ... 64150 65633 18843]


In [7]:
solrad_col_names = ["dni_jan_me", "dni_feb_me", "dni_mar_me", "dni_apr_me", "dni_may_me", "dni_jun_me",
                    "dni_jul_me", "dni_aug_me", "dni_sep_me", "dni_oct_me", "dni_nov_me", "dni_dec_me"]

In [8]:
new = hrus[solrad_col_names]

In [9]:
new.tail()

,dni_jan_me,dni_feb_me,dni_mar_me,dni_apr_me,dni_may_me,dni_jun_me,dni_jul_me,dni_aug_me,dni_sep_me,dni_oct_me,dni_nov_me,dni_dec_me
114953,114.202190,146.516835,179.218250,222.997086,240.846026,259.356323,306.073085,269.155830,239.265499,163.882154,127.888121,107.309903
114954,116.910396,155.363572,190.271046,211.189573,218.493133,262.913941,334.356097,293.992769,252.081908,174.155373,133.079809,103.451310
114955,100.200522,130.797801,165.890174,221.951858,237.911750,274.295443,350.563980,296.977183,250.828283,176.225818,124.652525,94.911594
114956,106.988632,148.589287,178.983503,223.445490,243.563589,281.699072,354.641561,307.093563,247.162792,177.625283,130.361148,109.739219
114957,128.077367,176.159291,217.490094,243.767839,227.622825,238.529215,268.132795,244.427464,196.997736,137.297803,98.566210,104.569645


In [10]:
solrad_targets = new.values

In [11]:
print(solrad_targets)

ihru = 2000
imon = 6

print(solrad_targets[ihru,imon])

[[182.20148911 183.51872029 191.96332414 ... 229.74948359 196.08214547
  166.2665358 ]
 [182.0873538  182.82732109 190.77398132 ... 227.66198595 193.96723928
  164.80365733]
 [183.55624823 184.54553207 191.84952426 ... 228.18403367 194.83623904
  165.23211128]
 ...
 [100.20052231 130.79780146 165.89017402 ... 176.22581824 124.65252498
   94.91159392]
 [106.98863186 148.58928652 178.98350311 ... 177.62528297 130.36114766
  109.73921902]
 [128.07736674 176.15929126 217.490094   ... 137.29780348  98.5662098
  104.56964488]]
272.6455278722463


In [12]:
# Free memory that is not used anymore.
del(new)
del(hrus)